<a href="https://colab.research.google.com/github/shrutika1202/Music-recommender/blob/main/MusicRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tekore as tk

In [4]:
import pandas as pd
from tqdm import tqdm
import time


In [10]:
def authorize():
 CLIENT_ID = '018d93f2ca0d444a91a17019923c7204'
 CLIENT_SECRET = '1f560bde80154336a6da253537ce0c3e'
 app_token = tk.request_client_token(CLIENT_ID, CLIENT_SECRET)
 return tk.Spotify(app_token)

In [11]:
# Authorize and call access object "sp"
sp = authorize()

# Get all genres
genres = sp.recommendation_genre_seeds()

# Set number of recommendations per genre
n_recs = 100

# Initiate a dictionary with all the information you want to crawl
data_dict = {"id":[], "genre":[], "track_name":[], "artist_name":[],
             "valence":[], "energy":[]}

In [ ]:
for g in tqdm(genres):
    
    # Get n recommendations
    recs = sp.recommendations(genres = [g], limit = n_recs)
    # json-like string to dict
    recs = eval(recs.json().replace("null", "-999").replace("false", "False").replace("true", "True"))["tracks"]
    
    # Crawl data from each track
    for track in recs:
        # ID and Genre
        data_dict["id"].append(track["id"])
        data_dict["genre"].append(g)
        # Metadata
        track_meta = sp.track(track["id"])
        data_dict["track_name"].append(track_meta.name)
        data_dict["artist_name"].append(track_meta.album.artists[0].name)
        # Valence and energy
        track_features = sp.track_audio_features(track["id"])
        data_dict["valence"].append(track_features.valence)
        data_dict["energy"].append(track_features.energy)
        
        # Wait 0.2 seconds per track so that the api doesnt overheat
        time.sleep(0.2)

In [31]:
a=len(data_dict["id"])
b=len(data_dict["genre"])
c=len(data_dict["track_name"])
d=len(data_dict["artist_name"])
e=len(data_dict["valence"])
f=len(data_dict["energy"])

g=list(data_dict["genre"])
print(g)
print(a,b,c,d,e,f)

print(data_dict["id"][a-1])

print(data_dict["id"][len(data_dict["id"])-1])
# data_dict = {"id":[], "genre":[], "track_name":[], "artist_name":[],
#              "valence":[], "energy":[]}

['acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'acoustic', 'ac

In [32]:
df = pd.DataFrame(data_dict)

In [33]:
df.drop_duplicates(subset = "id", keep = "first", inplace = True)
df.to_csv("valence_arousal_dataset.csv", index = False)

In [34]:
def distance(p1, p2):
    distance_x = p2[0]-p1[0]
    distance_y = p2[1]-p1[1]
    distance_vec = [distance_x, distance_y]
    norm = (distance_vec[0]**2 + distance_vec[1]**2)**(1/2)
    return norm

In [4]:
import pandas as pd
import random
import numpy as np
from numpy.linalg import norm

In [ ]:
df = pd.read_csv("valence_arousal_dataset.csv")

In [38]:
df["mood_vec"] = df[["valence", "energy"]].values.tolist()


In [39]:
sp = authorize()


In [1]:
def recommend(track_id, ref_df, sp, n_recs = 5):
    
    # Crawl valence and arousal of given track from spotify api
    track_features = sp.track_audio_features(track_id)
    track_moodvec = np.array([track_features.valence, track_features.energy])
    
    # Compute distances to all reference tracks
    ref_df["distances"] = ref_df["mood_vec"].apply(lambda x: norm(track_moodvec-np.array(x)))
    # Sort distances from lowest to highest
    ref_df_sorted = ref_df.sort_values(by = "distances", ascending = True)
    # If the input track is in the reference set, it will have a distance of 0, but should not be recommendet
    ref_df_sorted = ref_df_sorted[ref_df_sorted["id"] != track_id]
    
    # Return n recommendations
    return ref_df_sorted.iloc[:n_recs]

In [ ]:
mad_world = "3JOVTQ5h8HGFnDdp4VT3MP"
recommend(track_id = mad_world, ref_df = df, sp = sp, n_recs = 5)